In [1]:
# Task:
# An SVD Recommender that predicts the rating a user will give to a movie
# based on the user's own ratings and other users' rating data.

# Use only 'rating' as the data, avoid 'tags' and 'genre'

# 80/20, train/test split. Additionally, do a temporal split. 

In [2]:
# imports
import pandas as pd
from numpy.linalg import svd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import math
import random
import copy

In [3]:
eps = 1.0e-6

def cosine_similarity(v,u):
    return (v @ u.T)/ (np.linalg.norm(v) * np.linalg.norm(u) + eps)

def rmse_function(v, u):
    return np.sqrt(np.mean((v - u) ** 2))
    
def squared_distance(v, u):
    v = np.array(v)
    u = np.array(u)
    return np.sum((v[0] - u[0]) ** 2)

# Build a pivot table with movieIds as columns 
# and users as rows
def create_pivot_from_df(df, index, columns, values, reindex_columns):
    # Create the pivot matrix with the training data
    rating_matrix = df.pivot(index=index, columns=columns, values=values)

    # Reindex the columns of the pivot matrix to include all movie IDs
    rating_matrix = rating_matrix.reindex(columns=reindex_columns, fill_value=0)

    # Fill NaN values with 0
    rating_matrix = rating_matrix.fillna(0)
    
    display(rating_matrix.head())
    matrix = rating_matrix.values
    matrix = np.matrix(matrix)
    return matrix

# K-means RMSE Clustering 
# select k random data points (user indexes in u) as initial cluster centers C_1, ..., C_k

def initialize_centroids(data_points, k, seed = None):
    if seed is not None:
        random.seed(seed)
        
    # Select k random indexes from the data points list
    centroid_indexes = random.sample(range(data_points.shape[0]), k)
    
    # Construct the list of centroids using the selected indexes
    centroids = [copy.deepcopy(data_points[i]) for i in centroid_indexes]
    
    return centroids

def assign_point_to_centroid(user_index, point, centroid_index, assignments):
    if centroid_index not in assignments:
        assignments[centroid_index] = [(user_index, point)]
    else:
        assignments[centroid_index].append((user_index, point))

def update_centroid_position(centroid, new_position):
    centroid[:] = new_position
    
def average_point(points):
    # Extract the points (user points) from the tuples
    user_points = [point[1] for point in points]
    
    # Convert the list of points to a NumPy array
    points_array = np.array(user_points)

    # Compute the mean along each dimension
    average = np.mean(points_array, axis=0)
    
    return average

def k_means(u, k, max_iterations, seed, threshold):
    assignments = {}
    num_datapoints = u.shape[0]

    centroids = initialize_centroids(u, k, seed)
    
    for iteration in range(1, max_iterations):
        assignments = {}
        
        for index, _ in enumerate(centroids):
            assignments[index] = []
            
        non_converged_centroids = copy.deepcopy(k)
        
        # for each p (user) in u, map p_i to its nearest cluster center C_j 
        for user_index, p in enumerate(u):
            # Find the closest centroid to the current data point            
            closest_centroid_index, _ = max(enumerate(centroids), key=lambda c: cosine_similarity(p, c[1]))
            
            assign_point_to_centroid(user_index, p, closest_centroid_index, assignments)
        
        for index, centroid in enumerate(centroids):
            # Update centroid position by taking the mean of assigned data points
            
            assigned_points = assignments[index]  # get all data points assigned to a centroid
                
            if assigned_points:
                new_position = average_point(assigned_points)
                 # calculate Euclidean distance

                distance = np.linalg.norm(centroid - new_position)
                
                if distance < threshold:
                    non_converged_centroids -= 1

                update_centroid_position(centroid, new_position)
            
        if (non_converged_centroids == 0):
            print("iter: ", iteration, " ******* CONVERGED ******* ", end="")
            break
    
    total_rmse = 0
    for centroid_index, data_points in assignments.items():
        squared_distances = 0

        for point_index, point in enumerate(data_points):
            # calculate squared distancs of each point to its centroid
            squared_distances += squared_distance(point[1], centroids[centroid_index])
        
        # divide the sum of squared distances with the number of points in the centroid
        mse = squared_distances / len(centroids[centroid_index])

        # take the square root of mse to get rmse
        rmse = np.sqrt(mse)
        total_rmse += rmse
    
    # Calculate the average RMSE
    average_rmse = total_rmse / len(assignments)
    print("average_rmse: ", average_rmse)
    
    centroids_and_assignments = np.array([(centroids[i], assignments[i]) for i in range(len(centroids))], dtype=object)

    return centroids_and_assignments

# Generate similarities to all others points for a single point
def get_similarities(target_point, neighbors):
    similarities = []

    for index, neighbor in enumerate(neighbors):
        similarity = cosine_similarity(target_point, neighbor[1])
        similarities.append((neighbor, similarity))
    
    return similarities

def predict_ratings(centroid_and_assignment, ratings_matrix, k=5):
    predictions = []
    
    # Calculate cosine similarity between the centroid and its neighbors
    similarities = get_similarities(centroid_and_assignment[0], centroid_and_assignment[1])

    # make predictions for every movie
    for movie_index in range(ratings_matrix.shape[1]):

        # Calculate weighted average of ratings from nearest neighbors
        weighted_ratings = 0
        total_similarity = 0
        for neighbor_index, similarity in enumerate(similarities):

            neighbor_similarity = similarity[1]
            neighbor_rating = ratings_matrix[similarity[0][0], movie_index]

            if neighbor_rating != 0:  # Ignore if neighbor hasn't rated the movie
                weighted_ratings += neighbor_similarity * neighbor_rating
                total_similarity += neighbor_similarity

                #print("neighbor_similarity: ", neighbor_similarity)
                #print("neighbor_rating: ", neighbor_rating)
                #print("weighted rating: ", neighbor_similarity * neighbor_rating)
                #print("total_similarity: ", total_similarity)
                #print()
                #break

        # Predict the rating for the target user
        if total_similarity != 0:
            predictions.append(float(weighted_ratings / total_similarity))
        else:
            predictions.append(float(0))  # In case none of the nearest neighbors have rated the movie
    
    return predictions

def find_closest_centroid(data_point, centroids):
    closest_centroid_distance = np.inf
    closest_centroid_index = None
    
    for centroid_index, centroid in enumerate(centroids):
        distance = squared_distance(data_point, centroid)

        if distance < closest_centroid_distance:
            closest_centroid_distance = distance
            closest_centroid_index = centroid_index
            
    return closest_centroid_index

# evaluate 
def generate_prediction_set(rating_matrix, centroids_and_assignments, centroid_ratings):
    rating_array = np.array(rating_matrix)
    
    prediction_matrix = np.full((rating_matrix.shape[0], rating_matrix.shape[1]), 0)

    centroids = [centroid for centroid, _ in centroids_and_assignments]
    
    for p_index, p in enumerate(rating_matrix):
        # Generate vector of new user to latent factor
        p_result = p * Vr * np.linalg.inv(Sr)
        
        # Find closest centroid
        closest_centroid_index = find_closest_centroid(p_result, centroids)
        
        # Replace empty user predictions with centroid's predictions
        prediction_matrix[p_index] = centroid_ratings[closest_centroid_index]
    
    return prediction_matrix

def calculate_metrics_for_bin(actual_ratings, predicted_ratings, bin_value):
    # Create a mask for non-zero actual ratings
    non_zero_mask = (actual_ratings == 0)
        
    # Apply the mask to predicted ratings
    masked_predicted_ratings = np.ma.array(predicted_ratings, mask=non_zero_mask)
    masked_predicted_ratings = np.ma.filled(masked_predicted_ratings, fill_value=0)

    # Convert actual and predicted ratings to binary labels based on bin_value
    actual_labels = (actual_ratings == bin_value)
    predicted_labels = (masked_predicted_ratings == bin_value)

    # Calculate true positives, false positives, false negatives
    true_positives = np.sum(actual_labels & predicted_labels)
    false_positives = np.sum((~actual_labels) & predicted_labels)
    false_negatives = np.sum(actual_labels & (~predicted_labels))

    # Calculate precision
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0

    # Calculate recall
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0

    # Calculate F1-score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return precision, recall, f1_score

In [12]:
# read data
ratings = 'data/movielens-latest-small/ratings.csv'

# to dataframes
df_ratings = pd.read_csv(ratings)

# inspect them
display('Ratings')
display(df_ratings.head())

'Ratings'

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
# 80/20 random train/test split
all_movie_ids = np.sort(df_ratings['movieId'].unique())

df_train, df_test = train_test_split(df_ratings, test_size=0.2, random_state=1)

df_train = df_train.sort_values(by='timestamp')

# temporal split
train_time_series = df_ratings
test_time_series = df_ratings

tscv = TimeSeriesSplit(n_splits=2, test_size=20000)
for i, (train_index, test_index) in enumerate(tscv.split(df_ratings)):
    if i == 0:
        continue
    
    print(f"TimeSeriesSplit fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    print()
    
    train_time_series = df_ratings.iloc[train_index]
    test_time_series = df_ratings.iloc[test_index]
    
    print("train_time_series.shape :", train_time_series.shape)
    print("train_time_series head, tail:")
    display(train_time_series.head())
    display(train_time_series.tail())
    
    print("test_time_series.shape :",test_time_series.shape)
    print("test_time_series head, tail:")
    display(test_time_series.head())
    display(test_time_series.tail())

TimeSeriesSplit fold 1:
  Train: index=[    0     1     2 ... 80833 80834 80835]
  Test:  index=[ 80836  80837  80838 ... 100833 100834 100835]

train_time_series.shape : (80836, 4)
train_time_series head, tail:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


,userId,movieId,rating,timestamp
80831,509,117529,3.5,1435996029
80832,509,118997,2.5,1436000441
80833,509,122892,4.0,1435997969
80834,509,126142,4.0,1435998617
80835,509,129229,2.0,1435997905


test_time_series.shape : (20000, 4)
test_time_series head, tail:


,userId,movieId,rating,timestamp
80836,509,130073,4.0,1435997996
80837,509,133419,5.0,1435997941
80838,509,136838,2.5,1435998776
80839,509,136840,3.5,1435998963
80840,510,7,1.0,1141158812


,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [14]:
# train_matrix contains 80% of random data points in the entire 100k reviews
train_matrix = create_pivot_from_df(df_train, "userId", "movieId", "rating", all_movie_ids)
test_matrix = create_pivot_from_df(df_test, "userId", "movieId", "rating", all_movie_ids)

# matrix_test_time_series contains the chronological last 20k data points of the entire 100k reviews
matrix_train_time_series = create_pivot_from_df(train_time_series, "userId", "movieId", "rating", all_movie_ids)
matrix_test_time_series = create_pivot_from_df(test_time_series, "userId", "movieId", "rating", all_movie_ids)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
510,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
512,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
513,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Singular value decomposition. Source: https://www.kaggle.com/code/vincentman0403/recommendation-example-by-svd
U, S, VT = np.linalg.svd(train_matrix, full_matrices=False)

V = VT.T
Sigma = np.diag(S)
print('U shape = ', U.shape)
print('VT shape = ', VT.shape)
print('S shape = ', S.shape)
print('Sigma = \n', Sigma)

# Use first 2 singular values
r = 2

# Get approximate U, Sigma, VT
Ur = U[:, :r]
Sr = Sigma[:r, :r]
Vr = V[:, :r]
print('Ur(matrix of user to latent factor), shape = ', Ur.shape)
print('Sr(matrix of singular values), shape = ', Sr.shape)
print('Vr(matrix of item to latent factor), shape = ', Vr.shape)

Sr = Sigma[:r, :r]
Vr = V[:, :r]

U shape =  (610, 610)
VT shape =  (610, 9724)
S shape =  (610,)
Sigma = 
 [[430.73584462   0.           0.         ...   0.           0.
    0.        ]
 [  0.         189.33366799   0.         ...   0.           0.
    0.        ]
 [  0.           0.         156.70901141 ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   3.43523313   0.
    0.        ]
 [  0.           0.           0.         ...   0.           3.2973898
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    3.06286843]]
Ur(matrix of user to latent factor), shape =  (610, 2)
Sr(matrix of singular values), shape =  (2, 2)
Vr(matrix of item to latent factor), shape =  (9724, 2)


In [74]:
max_iterations = 50
threshold = 1e-5

# use specific start conditions
k = 10
seed = 0

centroids_and_assignments = k_means(Ur, k, max_iterations, seed, threshold)


average_rmse:  0.31125231607072845


In [75]:
# Generate matrix for centroid predictions
centroid_ratings = []

for index, centroid_and_assignment in enumerate(centroids_and_assignments):
    centroid_ratings.append(predict_ratings(centroid_and_assignment, ratings_matrix = train_matrix))

centroid_ratings = np.array(centroid_ratings)
print("centroid_ratings.shape:", centroid_ratings.shape)
#print("centroid_ratings:", centroid_ratings)

centroid_ratings.shape: (10, 9724)


In [76]:
prediction_matrix = generate_prediction_set(test_matrix, centroids_and_assignments, centroid_ratings)
#display(prediction_matrix)

In [82]:
bin_metrics = []

for bin_value in range(1, 6):    
    precision, recall, f1_score = calculate_metrics_for_bin(test_matrix, prediction_matrix, bin_value)
    bin_metrics.append({"Bin value": bin_value, "Precision": precision, "Recall": recall, "F1-score": f1_score})
    #print(f'{"Bin value:":<10}{bin_value:10}')
    #print(f'{"Precision:":<10}{round(precision, 2):10}')
    #print(f'{"Recall:":<10}{round(recall, 2):10}')
    #print(f'{"F1-score:":<10}{round(f1_score, 2):10}')
    #print()

average_precision = sum(metric["Precision"] for metric in bin_metrics) / len(bin_metrics)
average_recall = sum(metric["Recall"] for metric in bin_metrics) / len(bin_metrics)
average_f1_score = sum(metric["F1-score"] for metric in bin_metrics) / len(bin_metrics)

# Display the averages
print(f'{"Average precision:":<20}{round(average_precision, 2):10}')
print(f'{"Average recall:":<20}{round(average_recall, 2):10}')
print(f'{"Average F1-score:":<20}{round(average_f1_score, 2):10}')
print()

Average precision:        0.18
Average recall:           0.19
Average F1-score:         0.16



In [49]:
# Find converging start conditions, and lowest rmse
def test_range_seed_k():
    max_iterations = 50
    threshold = 1e-5

    for seed in range(0, 1):
        print("seed:",seed," ",end="")

        for k in range(3, 10): 
            print("k:",k," ",end="")
            k_means(u, k, max_iterations, seed, threshold)